# 「作法一」將爬下來的資料存進MongoDb

In [2]:
import requests
from datetime import datetime
import pandas as pd
from pymongo import MongoClient

In [33]:
#利用 Facebook Graph API Explorer 取得 token
#https://developers.facebook.com/tools/explorer
token = 'EAACEdEose0cBAKcZAFh9OTBBMXrnhJjoZAgRwkiWN1gZAiY9EN0tRc4dP8WTvDx5aIAdXZBTf83yusEq9az3bVeSuqdxakM1xJcNzTeGsvhuyjTxAtZAaSZAaQ6QQopz9ZACrG4ygcewZCFRjF1fuTKMxCV58hNUZCZCU1VDaldTM9reFiQWQnXSjLZA6wLW5WOTRkfXJGC9VR0qwZDZD'

#將20個目標政治人物的粉專網址存在fanpage_urls
fanpage_urls = ['https://www.facebook.com/DoctorKoWJ/']

#取得粉絲專頁的id與名稱，並將其包成字典fanpage_dict
fanpage_dict = {}
for fanpage_url in fanpage_urls:
    resp = requests.get('https://graph.facebook.com/v2.12/{}?access_token={}&debug=all&format=json&method=get&pretty=0&suppress_http_code=1'
                       .format(fanpage_url,token))
    fanpage_json = resp.json()
    fanpage_dict[fanpage_json['id']] = fanpage_json['name']

#建立一個空的list          
posts = []

#使用for迴圈依序讀取粉絲頁的資訊，並使用.format()將fanpage_id與token傳入{}裡
#觀察發現Facebook Graph API回傳的data內最多包含100則貼文，故 limit = 100
for fanpage_id in fanpage_dict:
    fanpage_name = fanpage_dict[fanpage_id]
    resp = requests.get('https://graph.facebook.com/v2.9/{}/posts?limit=100&access_token={}'.format(fanpage_id, token))
    
    #使用while迴圈去取得所有的資料，並利用next翻頁
    while resp.json(): 
        for post_data in resp.json()['data']:
        # 利用ternary conditional operator判斷每個post是否含有'story'或'message'這些key，若無'story'則為純文字，無'message'則無文字
                story = post_data['story'] if 'story' in post_data else '純文字'
                message = post_data['message'] if 'message' in post_data else story
                created_time = datetime.strptime(post_data["created_time"].split('+')[0], "%Y-%m-%dT%H:%M:%S")
                post_id = post_data['id']
                posts.append({'fanpage_name':fanpage_name, 'message':message, 'created_time':created_time, 'story':story, 'id':post_id})
        if 'next' in resp.json()['paging']:
            resp = requests.get(resp.json()['paging']['next'])
        else:
            break

#最後將posts轉換成pd.DataFrame
posts_df = pd.DataFrame([post.values()for post in posts], columns=['粉絲專頁', '發文內容', '發文時間', '發文類型', 'post_id'])
posts_df

#將dataframe寫出為json，並存進MongoDb
client = MongoClient("mongodb://127.0.0.1:27017")
db = client['<db_name>']
collection = db["<collecion_name>"] 
collection.insert_many(posts)

In [31]:
posts_df

,粉絲專頁,發文內容,發文時間,發文類型,post_id
0,柯文哲,今天來開平餐飲學校學怎麼煎牛排，用英文學，希望最後有熟。\n\n#創新實驗教育\n臺北市教育...,2018-03-09 02:19:54,柯文哲 was live.,136845026417486_1263255723776405
1,柯文哲,【二備金不等於市長小金庫】\n\n目前的政府預算制度，大概在5、6月時就編列預算了，如果議會...,2018-03-08 09:00:00,純文字,136845026417486_1262597930508851
2,柯文哲,不同個性、不同能力的孩子，卻接受同樣一套教育，考同一張考卷，這並不合理。因此，「實驗教育」是...,2018-03-06 11:55:36,純文字,136845026417486_1260825347352776
3,柯文哲,今年的台北燈節落幕了，雖然資源有限，但市府同仁和策展單位都拼了，每一盞燈、每一座裝置藝術，我...,2018-03-05 11:00:00,純文字,136845026417486_1259930147442296
4,柯文哲,今晚，台北燈節就要告一段落，榮耀歸於所有辛苦的無名英雄們。\n\n很多公務員為了彩排昨晚的燈...,2018-03-04 11:58:40,柯文哲 added 4 new photos.,136845026417486_1259131887522122
5,柯文哲,過去公園裡的遊戲場，我們以為把跟罐頭一樣千篇一律的遊具擺上去就可以了，事實上玩這些遊具身體不...,2018-03-04 04:32:20,純文字,136845026417486_1258854180883226
6,柯文哲,今天晚上七點半，搭捷運來北門廣場，打開你的手機燈光，帶著你的小提燈，跟我一起在中華路點亮一隻...,2018-03-03 11:30:08,柯文哲 was live — at 北門廣場.,136845026417486_1258281140940530
7,柯文哲,【用燈節拉近世代距離】\n \n如果說世大運成功提升了台北人的光榮感，那麼在西區舉辦台北燈節...,2018-03-02 10:15:36,柯文哲 added 4 new photos.,136845026417486_1257206147714696
8,柯文哲,決定政策有三個要素：民意、專業、價值。進入網路時代之後，讓直接民主變成可能，iVoting就...,2018-03-01 09:43:11,純文字,136845026417486_1256313571137287
9,柯文哲,蔡總統、賴院長、二二八受難者與家屬、各位來賓、媒體朋友，大家好。\n\n1947年發生的二二...,2018-02-28 07:45:11,純文字,136845026417486_1255447907890520


# 「實驗」爬取comment

In [34]:
posts

[{'_id': ObjectId('5aa49e31c54d2d327672c715'),
  'created_time': datetime.datetime(2018, 3, 9, 2, 19, 54),
  'fanpage_name': '柯文哲',
  'id': '136845026417486_1263255723776405',
  'message': '今天來開平餐飲學校學怎麼煎牛排，用英文學，希望最後有熟。\n\n#創新實驗教育\n臺北市教育局\nKPCS 開平餐飲學校\n\n---\n台北市創新實驗教育4分鐘紀錄片\nhttps://goo.gl/NfvbWT',
  'story': '柯文哲 was live.'},
 {'_id': ObjectId('5aa49e31c54d2d327672c716'),
  'created_time': datetime.datetime(2018, 3, 8, 9, 0),
  'fanpage_name': '柯文哲',
  'id': '136845026417486_1262597930508851',
  'message': '【二備金不等於市長小金庫】\n\n目前的政府預算制度，大概在5、6月時就編列預算了，如果議會準時通過，理論上隔年的1月1日就可以開始執行。\n\n但是如果在6月之後突然有一個政策想執行，在目前的預算制度下，要等明年再編預算，最快要等到後年的1月1日以後才能執行。\n\n所以第二預備金的目的，在於如果政府有臨時業務，不用再等一年半的時間，可先用第二預備金來支應來提高施政效率。另外，極端氣候跟新興傳染疾病的威脅，也需要二備金來應急。\n\n過去一年來，我們利用二備金增加老人共餐據點，擴大敬老卡使用範圍（可搭捷運、騎Youbike），整修北投中心新村，補助免費接種兒童輪狀病毒疫苗、老人肺炎鏈球菌疫苗，辦理購物塑膠袋與專用垃圾袋兩袋合一，建置台北車站智慧化系統，增購動態智慧型公車站牌，設立幹線公車⋯等等。\n\n也許動支上還有可以改進的地方，但這些成果，代表我們很謹慎的使用每一筆人民的血汗錢。\n\n第二預備金不應該像過去一樣，被認為是市長個人的小金庫，即使沒有議會的事前監管，也要自我要求謹慎使用。使用二備金的態度，正是建立新的政治文化的開始。\n\n#建立全新政治

In [57]:
from dateutil.parser import parse


#在Facebook Graph API Exploer取得token以及粉絲專頁的ID
token = 'EAACEdEose0cBAMF7QxihZCg0Dq1OW1nQaZCD0YqzPZAtLd2GuzLeuVdFwFjXiZBsCExhiurtwxZBTwwEGwvDa4ujh4fPaUxDChaZACYtZAldRD6aTh3ojZADddZCXGolBCWG4km96gjxHQKGHpeumHFA3fJ87vAWENW3qjv4vtgZAPPlZCdZBjcdQnRluEjkE0niP80CGoM83N9CGwZDZD' 
fanpage_id = '136845026417486'

#目標頁面
resp = requests.get('https://graph.facebook.com/v2.12/{}/posts?limit=100&access_token={}'.format(fanpage_id, token))
page = 1  

#建立一個空的list          
comments = []

#API最多一次呼叫100筆資料，因此使用while迴圈去翻頁取得所有的文章
#enumerate is useful for obtaining an indexed list:(0, seq[0]), (1, seq[1]), (2, seq[2]), ...
while resp.json(): 
    for index, post_data in enumerate(resp.json()['data']):
        print('正在爬取第{}頁，第{}篇文章'.format(page, index + 1))
        
        
        #判斷是否為發文，是則開始蒐集按讚ID(不需判斷..by nat)
        for post in posts:
            resp_post = requests.get("https://graph.facebook.com/v2.6/%s?fields=comments{created_time, id, message, likes.limit(0).summary(True), from}&access_token=%s"%(post['id'],token))
            
            #判斷按讚人數是否超過1000人，若超過則需要翻頁擷取；當沒有人按讚時，按讚人名與ID皆為NONE
        while True:
            for comment in resp_post.json()["data"]:
                comment["created_time"] = datetime.strptime(comment["created_time"], "%Y-%m-%dT%H:%M:%S+0000")
                comment["likes"] = comment["likes"]["summary"]["total_count"]
                comment["post_id"] = post_id
                comments.append(comment)
                print("  finish {}".format(comment["id"]))

            try:
                if "next" in resp_post.json()["paging"]:
                    next_page = resp_post.json()["paging"]["next"]
                    resp_post = requests.get(next_page)
                else:
                    break
            except:
                break

#             try:
#                 if 'next' not in resp_post.json()['paging']:
#                     for likes in resp_post.json()['data']:
#                         comments.append([post_data['id'], post['message'], parse(post['created_time']).date(), likes['id'], likes['name']])                
#                 elif 'next' in res_post.json()['paging']:
#                     while 'paging' in resp_post.json():
#                         for likes in resp_post.json()['data']:
#                             comments.append([post['id'], post['message'], parse(post['created_time']).date(), likes['id'], likes['name']])
#                         if 'next' in resp_post.json()['paging']:
#                             resp_post = requests.get(res_post.json()['paging']['next'])
#                         else:
#                             break
#             except:
#                 comments.append([post['id'], post['message'], parse(post['created_time']).date(), "NONE", "NONE"])

    if 'next' in res.json()['paging']:                
        res = requests.get(res.json()['paging']['next'])
        page += 1
    else:
        break
        
print('爬取結束!')

正在爬取第1頁，第1篇文章


KeyboardInterrupt: 

In [ ]:
resp_post.json()

In [ ]:
# This will bring the created_time, id, comment_message, total_count of likes, from(the name & id of whom left the comment)
post_id='136845026417486_1263255723776405'
requests.get("https://graph.facebook.com/v2.6/%s?fields=comments{created_time,id,message,likes.limit(0).summary(True),from}&access_token=%s"%(post_id,token)).json()

In [ ]:
# This will bring the total count of shares, comments and likes.
# (add limit(0) to removes lists of likes and comments and get only summary data)
# <post_id>?fields=comments.summary(true).limit(0), likes.summary(true).limit(0), shares
post_id='136845026417486_1263255723776405'
requests.get("https://graph.facebook.com/v2.6/%s?fields=comments.summary(true).limit(0), likes.summary(true).limit(0), shares&access_token=%s"%(post_id,token)).json()

# References: